In [11]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import pickle
import random
from tqdm import tqdm
import os

def primary_maker(null_data_path, t_name) : 
    null_df = pd.read_excel(null_data_path)
    primary = null_df[null_df['Prescription'] == t_name]['Primary']
    
    all_diseases_list = []
    
    # 2. 'diseases' section, delete (NaN)
    for item in primary.dropna():
        split_items = [name.strip() for name in item.split(',') if name.strip()]
        all_diseases_list.extend(split_items)
    
    unique_diseases_primary = sorted(list(set(all_diseases_list)))
    
    return unique_diseases_primary

def secondary_maker(null_data_path, t_name) : 
    null_df = pd.read_excel(null_data_path)
    secondary = null_df[null_df['Prescription'] == t_name]['secondary']
    
    
    all_diseases_list = []
    
    # 2. 'diseases' (NaN) deletion
    for item in secondary.dropna():
        split_items = [name.strip() for name in item.split(',') if name.strip()]
        all_diseases_list.extend(split_items)
    
    # 3. (option) delete duplication of disease
    unique_diseases_secondary = sorted(list(set(all_diseases_list)))
    return unique_diseases_secondary

# --- 1. prescription name settings ---
prescription_names = [
    "Gamisoyo-san", "Galgeun-tang", "Galgeunhaegi-tang", "Daeshiho-tang", "Banhabakchulcheonma-tang", "Banhasasim-tang",
    "Banhahubak-tang", "Bojungikgi-tang", "Saengmaek-san",
    "Sosiho-tang", "Socheongryong-tang", "Hyeonggaeyeongyo-tang",
    "Hwanglyeonhaedok-tang" 
]

null_data_path = r'C:\Users\seoku\Desktop\논문리비전\supplementary file\File S2.xlsx' 
data_path = r'C:\Users\seoku\논문관련자료'
figure_path = r'C:\analysis_output\histograms'
os.makedirs(figure_path, exist_ok=True)

SMDE_data_path = r'C:\Users\seoku\Downloads\SMDE.xlsx'
df = pd.read_excel(SMDE_data_path)


N_PERMUTATIONS = 10000

# select "primary", "secondary", "all" key
gold_standard_indications = {
    "Gamisoyo-san": {
        "primary": primary_maker(null_data_path, prescription_names[0]),
        "secondary": secondary_maker(null_data_path, prescription_names[0])
    },
    "Galgeun-tang": {"primary": primary_maker(null_data_path, prescription_names[1]), "secondary": secondary_maker(null_data_path, prescription_names[1])},
    "Galgeunhaegi-tang": {"all": primary_maker(null_data_path, prescription_names[2])},
    "Daeshiho-tang": {"all": primary_maker(null_data_path, prescription_names[3])},
    "Banhabakchulcheonma-tang": {"all": primary_maker(null_data_path, prescription_names[4])},
    "Banhasasim-tang": {"all": primary_maker(null_data_path, prescription_names[5])},
    "Banhahubak-tang": {"primary": primary_maker(null_data_path, prescription_names[6]), "secondary": secondary_maker(null_data_path, prescription_names[6])},
    "Bojungikgi-tang": {"primary": primary_maker(null_data_path, prescription_names[7]), "secondary": secondary_maker(null_data_path, prescription_names[7])},
    "Saengmaek-san": {"primary": primary_maker(null_data_path, prescription_names[8]), "secondary": secondary_maker(null_data_path, prescription_names[8])},
    "Sosiho-tang": {"primary": primary_maker(null_data_path, prescription_names[9]), "secondary": secondary_maker(null_data_path, prescription_names[9])},
    "Socheongryong-tang": {"all": primary_maker(null_data_path, prescription_names[10])},
    "Hyeonggaeyeongyo-tang": {"all": primary_maker(null_data_path, prescription_names[11])},
    "Hwanglyeonhaedok-tang": {"primary": primary_maker(null_data_path, prescription_names[12]), "secondary": secondary_maker(null_data_path, prescription_names[12])}
}

# whole disease list 
universe_of_all_diseases = list(df['Field_context'])

# --- 2. analyze function---
def get_predicted_diseases(graph):
    predicted = [node for node, data in graph.nodes(data=True) if data.get('Group') == 'disease']
    return predicted

def calculate_concordance(predicted_diseases, gold_standard_list):
    if not gold_standard_list: return 0.0
    matches = len(set(predicted_diseases).intersection(set(gold_standard_list)))
    return (matches / len(gold_standard_list)) * 100

#  main analyze loop 
print("--- Null model simulation ---")

for t_name in prescription_names:
    print(f"\n>>> {t_name} Analyzing...")
    
    try:
        with open(os.path.join(data_path, t_name + "_G.pkl"), 'rb') as f:
            G_real = pickle.load(f)
    except FileNotFoundError:
        print(f"!!! Warning: {t_name}_G.pkl, unable to find. skip.")
        continue

    real_predicted_diseases = get_predicted_diseases(G_real)
    indications = gold_standard_indications.get(t_name, {})
    
    num_diseases_to_predict = len(real_predicted_diseases)
    if num_diseases_to_predict == 0:
        print(f"!!! {t_name}: skip simulation, no predicted disease.")
        continue

    # make indication_types dictionary to analyze/visualize only existing data
    indication_types = {}
    if indications.get('primary'): indication_types['primary'] = indications['primary']
    if indications.get('secondary'): indication_types['secondary'] = indications['secondary']
    if indications.get('all'): indication_types['all'] = indications['all']

    # analyze each indication type 
    for ind_type, gold_standard_list in indication_types.items():
        # real concordance rate calculation
        real_concordance = calculate_concordance(real_predicted_diseases, gold_standard_list)
        
        # Null model simulation
        random_concordances = []
        for _ in range(N_PERMUTATIONS):
            random_predicted_diseases = random.sample(universe_of_all_diseases, k=num_diseases_to_predict)
            random_rate = calculate_concordance(random_predicted_diseases, gold_standard_list)
            random_concordances.append(random_rate)
            
        # calculating P-value and show output
        p_value = (sum(1 for r in random_concordances if r >= real_concordance) + 1) / (N_PERMUTATIONS + 1)
        num_gs_indications = len(gold_standard_list)
        # add null model average value
        null_mean_rate = pd.Series(random_concordances).mean()
        print(f"  - {ind_type.capitalize():<12} -> Real Rate: {real_concordance:6.2f}% (vs Null Mean: {null_mean_rate:6.2f}%), P-value: {p_value:.4f} (n={num_gs_indications})")
        #print(f"  - {ind_type.capitalize()} Indications -> Real Rate: {real_concordance:.2f}%, P-value: {p_value:.4f}")

        # visualizing histogram
        plt.figure(figsize=(10, 6))
        plt.hist(random_concordances, bins=30, color='gray', alpha=0.7, label=f'Null Distribution (n={N_PERMUTATIONS})')
        plt.axvline(real_concordance, color='red', linestyle='--', linewidth=2, label=f'Observed Rate of real concordance ({real_concordance:.2f}%)')
        
        # displaying Gold Standard quantity 
        num_gs_indications = len(gold_standard_list)
        text_to_display = f'# of Gold Standard Indications: {num_gs_indications}'
        # add text, left-upper side
        plt.text(0.05, 0.95, text_to_display, 
                 transform=plt.gca().transAxes, 
                 fontsize=12, 
                 verticalalignment='top', 
                 bbox=dict(boxstyle='round,pad=0.5', fc='wheat', alpha=0.5))
        
        plt.title(f'Null Model for {t_name} - {ind_type.capitalize()} Indications\n(p-value = {p_value:.4f})')
        plt.xlabel('Concordance Rate (%)')
        plt.ylabel('Frequency')
        plt.legend()
        plt.grid(axis='y', linestyle='--', alpha=0.6)
        plt.savefig(os.path.join(figure_path, f'{t_name}_null_model_{ind_type}.png'))
        plt.close()

print("\n all simulation complete.")

--- Null model simulation ---

>>> Gamisoyo-san Analyzing...
  - Primary      -> Real Rate:  30.00% (vs Null Mean:   8.48%), P-value: 0.0441 (n=10)
  - Secondary    -> Real Rate:  42.86% (vs Null Mean:   7.16%), P-value: 0.0093 (n=7)

>>> Galgeun-tang Analyzing...
  - Primary      -> Real Rate:   0.00% (vs Null Mean:   7.43%), P-value: 1.0000 (n=5)
  - Secondary    -> Real Rate: 100.00% (vs Null Mean:   8.65%), P-value: 0.0866 (n=1)

>>> Galgeunhaegi-tang Analyzing...
  - All          -> Real Rate:  40.00% (vs Null Mean:  11.79%), P-value: 0.1093 (n=5)

>>> Daeshiho-tang Analyzing...
  - All          -> Real Rate:  66.67% (vs Null Mean:   9.23%), P-value: 0.0001 (n=12)

>>> Banhabakchulcheonma-tang Analyzing...
  - All          -> Real Rate:  60.00% (vs Null Mean:  17.17%), P-value: 0.0373 (n=5)

>>> Banhasasim-tang Analyzing...
  - All          -> Real Rate:  50.00% (vs Null Mean:   9.96%), P-value: 0.0002 (n=14)

>>> Banhahubak-tang Analyzing...
  - Primary      -> Real Rate:  66.67%